# Programming Assignment: 


1. На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)
2. Реализуйте два алгоритма рекомендаций:

    сортировка просмотренных id по популярности (частота появления в просмотренных),
    сортировка просмотренных id по покупаемости (частота появления в покупках).

3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие?           Обратите внимание на различие качества на обучающей и тестовой           выборке в случае рекомендаций по частотам покупки.



In [1]:
import numpy as np
import pandas as pd
from scipy import linalg
import collections


import numpy as np


**Входные данные**

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

**Важно:**

* Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
* Если товар не встречался в обучающей выборке, его популярность равна 0.  
* Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
* Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.

In [2]:
f = open("test.txt", "r")
test = f.read().splitlines() 
test[:5]

['6,7,8;', '13,14,15;', '22,23;', '28,29,30,31,32,33;', '40,41;']

In [3]:
f = open("train.txt", "r")
train = f.read().splitlines() 
train[:5]

['0,1,2,3,4,5;',
 '9,10,11,9,11,12,9,11;',
 '16,17,18,19,20,21;',
 '24,25,26,27,24;',
 '34,35,36,34,37,35,36,37,38,39,38,39;']

# Task №1
На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)


----
# 1
Каждая строчка в файлах - сессия. Читаем файлы построчно, укладываем данные о сессии в словарь с двумя полями: viewed и purchased. Значение в каждом поле - массив соответствующих id-шников. Все такие словари-сессии укладываем в свой массив для каждого файла, т.е. файл теперь представлен набором сессий, а сессия - словарем из двух массивов id-шников.

In [4]:
def data_preparation(path):
    f = open(path, "r")
    text = f.read().splitlines() 
    data = []

    for line in text:
        seen, bought = line.split(';')

        if bought != '':
            sess = {"seen" : seen.split(",") ,"bought" : bought.split(",")}
        else: 
            sess = {"seen" : seen.split(","), "bought" : []}
        data.append(sess)
    return data

In [5]:
test = data_preparation("test.txt")
train = data_preparation("train.txt")

# Task №2
Реализуйте два алгоритма рекомендаций:

* сортировка просмотренных id по популярности (частота появления в просмотренных),
* сортировка просмотренных id по покупаемости (частота появления в покупках).

## 2
Создаем два Counter'а, для viewed и purchased, в цикле скармливаем им id-шники из сессий файла train. Удобно скармливать сразу массив id-шников каждой сессии c помощью метода Counter.update. Counter крут еще и тем, что выдает 0 по умолчанию, т.е. когда к нему обращаются по ключу, которого в нем нет. Именно такое поведение и нужно в задании.

In [6]:
def calculate_frequency(data):
    counter_seen = collections.Counter()
    counter_bought = collections.Counter()
    for sess in data:
        counter_seen.update(sess.get("seen"))
        counter_bought.update(sess.get("bought"))
    return counter_seen, counter_bought

In [7]:
counter_seen, counter_bought = calculate_frequency(train)

## 3
Теперь можно удалить сессии, у которых массив purchased - пустой. Не .dropna, но тоже элементарно. А можно не удалять из самих выборок, а удалять из рассмотрения при расчете метрик, я так сделал.

In [8]:
def clear_dataset(data):
    data_clear = []
    for sess in data:
        if sess.get("bought") != []:
            data_clear.append(sess)
    return data_clear

In [9]:
train = clear_dataset(train)
test = clear_dataset(test)

## 4 
Теперь пишем функцию для сортировки viewed id-шников сессии, но тут можно обойтись без сложного ключа сортировки. Удаляем дублирующиеся id, сохраняя их изначальный порядок.  

In [10]:
def build_recommendations(data, counter_seen):
    rec_arr = []

    for i in range(len(data)):
        rec_dict = {}
        ids = data[i].get("seen")
        ids = list(set(ids))

        for id_value in ids:
            rec_dict.update({id_value: counter_seen.setdefault(id_value, 0)})
        rec_arr.append(collections.Counter(rec_dict).most_common())
    return rec_arr 

In [11]:
rec_arr = build_recommendations(test, counter_seen)

In [12]:
rec_arr[:5]

[[('63', 6), ('61', 2), ('68', 2), ('66', 2), ('59', 1), ('69', 0), ('70', 0)],
 [('158', 641), ('162', 318), ('160', 92), ('159', 81), ('161', 74)],
 [('204', 396), ('202', 66), ('203', 19), ('200', 18), ('201', 12)],
 [('371', 9), ('372', 5)],
 [('422', 60)]]

Теперь можно сортировать просто по частоте id-шника:

# Task №3
Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие?           Обратите внимание на различие качества на обучающей и тестовой           выборке в случае рекомендаций по частотам покупки.

Recall: Берем отсортированные в п.3 уникальные id в рамках сессии, обрезаем на k. Если массив оказался <k, то он таковым и остается.Ищем пересечение между этим массивом и массивом покупок в рамках текущей сессии. Делим на количество покупок в рамках текущей сессии


In [31]:
def metrics(k, rec_arr, sess_dict):
    precision = np.zeros((len(rec_arr), 1))
    recall = np.zeros((len(rec_arr), 1))
    for i in range(len(rec_arr)):
        sess_bought = rec_arr[i][:k]
        bought_id = [int(tup[0]) for tup in sess_bought]
        counter = 0

        ids = sess_dict[i].get("bought")
        

        #print(bought_id, ids, len(bought_id),"\n")

        for id_value in ids:
            id_value = int(id_value)
            if id_value in bought_id:
                counter += 1
            #print(counter)
        precision[i, 0] = counter /k
        recall[i, 0] = counter / len(ids)
        #print(ids, bought_id,"\t", counter, precision[i, 0], recall[i, 0])
        #precision += counter /k
        #recall += counter / len(bought_id)
        
    precision = np.mean(precision)#precision/ len(rec_arr) 
    recall = np.mean(recall) #recall/len(rec_arr)
    return precision, recall

In [15]:
def metric_to_file(filename, ar1, ap1, ar5, ap5):
    f=open(filename,'w')
    f.write(str(round(ar1,2))+" ")
    f.write(str(round(ap1,2))+" ")
    f.write(str(round(ar5,2))+" ")
    f.write(str(round(ap5,2))+" ")
    f.close()

In [17]:
k = 5
precision = 0
recall = 0
for i in range(len(rec_arr)):
    sess_bought = rec_arr[i][:k]
    bought_id = [int(tup[0]) for tup in sess_bought]
    counter = 0

    ids = test[i].get("bought")

    #print(bought_id, ids, len(bought_id),"\n")
    
    for id_value in ids:
        id_value = int(id_value)
        if id_value in bought_id:
            counter += 1
        #print(counter)
    precision += counter /k
    recall += counter / len(bought_id)

Precision: Берем отсортированные в п.3 уникальные id в рамках сессии, обрезаем на k.  Если массив оказался <k, то он таковым и остается.Ищем пересечение между этим массивом и массивом покупок в рамках текущей сессии. Делим на k просто

In [49]:
def summary(filename):
    # 1
    test = data_preparation("test.txt")
    train = data_preparation("train.txt")

    # 2
    counter_seen, counter_bought = calculate_frequency(train)

    # 3
    train = clear_dataset(train)
    test = clear_dataset(test)

    # 4
    rec_arr = build_recommendations(train, counter_seen)

    # 5

    precision_5, recall_5 = metrics(5, rec_arr, train)
    precision_1, recall_1 = metrics(1, rec_arr, train)
    print(round(recall_1, 2), round(precision_1, 2), round(recall_5, 2), round(precision_5, 2))
    # 6 
    metric_to_file(filename, recall_1, precision_1, recall_5, precision_5)

In [50]:
summary("answer_1.txt")

0.44 0.51 0.82 0.21


# 2


In [59]:
test = data_preparation("test.txt")
train = data_preparation("train.txt")

# 2
counter_seen, counter_bought = calculate_frequency(train)
len(counter_seen)

# 3
train = clear_dataset(train)
test = clear_dataset(test)

# 4
rec_arr = build_recommendations(train, counter_bought)

# 5

precision_5, recall_5 = metrics(5, rec_arr, train)
precision_1, recall_1 = metrics(1, rec_arr, train)
print(round(recall_1, 2), round(precision_1, 2), round(recall_5, 2), round(precision_5, 2))
# 6 
metric_to_file("answer_2.txt", recall_1, precision_1, recall_5, precision_5)

0.68 0.79 0.93 0.25


# 3

In [54]:

test = data_preparation("test.txt")
train = data_preparation("train.txt")

# 2
counter_seen, counter_bought = calculate_frequency(test)

# 3
train = clear_dataset(train)
test = clear_dataset(test)

# 4
rec_arr = build_recommendations(test, counter_bought)

# 5

precision_5, recall_5 = metrics(5, rec_arr, test)
precision_1, recall_1 = metrics(1, rec_arr, test)
print(round(recall_1, 2), round(precision_1, 2), round(recall_5, 2), round(precision_5, 2))
# 6 
metric_to_file("answer_4.txt", recall_1, precision_1, recall_5, precision_5)

0.67 0.78 0.92 0.25


# 4

In [55]:
test = data_preparation("test.txt")
train = data_preparation("train.txt")
# 2
counter_seen, counter_bought = calculate_frequency(test)

# 3
train = clear_dataset(train)
test = clear_dataset(test)

# 4
rec_arr = build_recommendations(test, counter_seen)

# 5

precision_5, recall_5 = metrics(5, rec_arr, test)
precision_1, recall_1 = metrics(1, rec_arr, test)
print(round(recall_1, 2), round(precision_1, 2), round(recall_5, 2), round(precision_5, 2))
# 6 
metric_to_file("answer_3.txt", recall_1, precision_1, recall_5, precision_5)

0.43 0.5 0.81 0.21
